# Introduction à la manipulation et à l'analyse d'images en Python

- Auteurs : Jeyani George-Clement, Tristan Woh et William Amory  
- Université de Paris  

La manipulation et l'analyse d'images en python sont réalisées à l'aide du module [scikit-image](https://scikit-image.org/), qui est un des nombreux projets de base de [scipy stack](https://www.scipy.org/docs.html). Il existe d'autres modules permettant de réaliser de l'analyse d'image tel que [Pillow](https://python-pillow.org/), [SimpleITK](https://simpleitk.org/), et tant d'autres. 

Ce module est un des plus utilisé pour la manipulation et l'analyse d'image en python. Il comptabilise 4626 étoiles sur leurs repository GitHub, leur dernier commit est très récent. Ce qui démontre que le module est en constante évolution. De plus, il y a 136 issues ouvertes tandis que 1992 sont fermées nous démontrant que la communauté Python autour de ce module est très active. La toute première version de scikit-image remonte à août 2009. La version actuelle, et celle utilisée dans cette démonstration de l'utilisation du module, est la version 0.18.3. Le dépot GitHub du projet scikit-image est consultable [ici](https://github.com/scikit-image).

Il est principalement utilisé pour réaliser de la segmentation, du filtrage, de la transformation que nous allons aborder au cours de cette introduction. L'installation du module à partir des gestionnaires de packages [pip](https://pypi.org/) et [conda](https://anaconda.org/) à l'aide des commandes suivantes.  

```bash
# installation with conda :
conda install -c conda-forge scikit-image

# installation with pip :
## Debian/Ubuntu :
sudo apt-get install python-skimage
## OSX :
pip install scikit-image
```

## Contexte

La miscroscopie est une technologie qui permet de stimuler les découvertes en biologie. Cette technologie n'est pas "nouvelle" puisque le premier microscope, inventé par Antonie Van Leeuwenhoek, date de la fin du XVIIe siècle. De plus, depuis ces 15 dernières années l'évolution de la microscopie à fait de très grands pas en avant permettant d'aller au-delà de la limite de la diffraction de la lumière. Comme les microscopes photoniques confocales qui permet de localiser le faisceau de photon sur une région précise de la cellule, réduisant ainsi le bruit [ajouter image avec et sans bruit].  

Ces dernières années, un nouveau type de microscope a vue le jour, il s'agit de la microscopie à super-résolution dit nanoscope qui va au-delà de la frontière de la diffraction de la lumière en descendant en-dessous des 200 nm de résolution [source].

Aujourd'hui beaucoup de résultats basés sur des images de microscopie demandent une analyse quantitative. Il est donc nécéssaire de réaliser l'interprétation quantitative du contenu des images en microscopie et de détecter automatiquement les objets ainsi que de réaliser des mesures sur ces objets. Tout comme en génétique l'imagerie génère énormément de données. On pourrait parler de "microscomique", comme on parle de génomique pour l'analyse de données massive en génétique ou de protéomique et métabolomique.




## Objectifs

L'objectif est d'explorer en surface le module scikit-image. Nous allons donc réaliser une segmentation de l'image (détection des objets de l'image). Nous montrerons comment fitrer les objets abérants. Nous réaliserons aussi une identification des noyaux, et déterminerons la correspondance cellulaire des fluorochromes. Ensuite nous réaliserons des mesures simples de la taille, la fluorescence, la gralunométrie des objets d'une image.

## Comment est codée informatiquement une image ?

Dans le cadre de notre projet, nous utiliserons uniquement des images numériques. Une image numérique, est une représentation matricielle de chaque pixel de l'image. Chaque pixel (indice dans la matrice) est une mesure du nombre de photons qui ont atteint une zone particulière du capteur de la caméra. Dans le cas d'une image en noir et blanc (niveaux de gris), chaque pixel est représenté par une valeur unique. Il s'agit soit d'un réel compris entre 0 et 1, soit d'un entier compris 0 et 255. Concernant les images en couleurs (RGB), chaque pixel est représenté par un tuple de 3 valeurs. Dont chaque position correspond à une couleur spécifique (rouge, vert et bleu). Tout commme une image en noir et blanc chaque valeur dans le tuple représentant le pixel correpond soit à un réel (entre 0 et 1), soit à un entier (entre 0 et 255).  
 

Le nombre de bits utilisé définit le nombre de couleur présente dans une image. Nous considérons que pour un nombre $n$ de bits nous aurons $2\times n$ couleurs dans notre image.  

Ci-dessous deux exemples de représentation d'une image :  

1. Dans le premier exemple nous avons créer artificiellement une image en noir et blanc à l'aide d'une matrice. Image que nous avons sauvegardé au format PNG.  

$\begin{equation}
\begin{pmatrix}
     1&1&1&1&1&1\\
     1&0.5&1&1&0.5&1\\
     1&0&1&1&0&1\\
     1&1&1&1&1&1\\
     1&0&0&0&0&1\\
     1&1&1&1&1&1
\end{pmatrix}~~~~
\text{Représentation matricielle de notre smiley en noir et blanc}
\end{equation}$

![smiley1](../img/smiley.png "Smiley en noir et blanc")

Pour arriver à ce résultat nous avons créé une matrice $6\times 6$ que nous avons remplis avec des valeurs réelles compris entre 0 et 1. Sachant que 1 correspond à un pixel "allumé" qui est associé à une couleur blanche et que 0 correspond à un pixel "éteint" qui est associé à la couleur noire. Si nous avions remplis notre matrice avec des entiers, chaque pixel serait compris entre 0 et 255. Sachant que la valeur 255 représente un pixel "allumé" associé à la couleur blanche.  
Le code Python permettant de générer cette image est le suivant :   

```python
import numpy as np

smiley = np.array([[1,1,1,1,1,1],
                  [1,0.5,1,1,0.5,1],
                   [1,0,1,1,0,1],
                  [1,1,1,1,1,1],
                  [1,0,0,0,0,1],
                  [1,1,1,1,1,1]])
plt.imshow(smiley, cmap = plt.get_cmap("gray"))
plt.savefig("smiley.png");
```
 

Ainsi, nous venons d'observer comment passer simplement d'un tableau contenant des valeurs à la création d'une image.  


2. Dans un second exemple, nous allons voir comment refaire cette même image en couleur. Pour faire cela nous utiliserons un tableau contenant le code couleur RGB par pixel. Chaque pixel correspond à un tuple contenant trois valeurs pour chacune des trois couleurs allant de 0 à 255. La même image pourra être générée à partir d'une matrice contenant des réels compris entre 0 et 1.  

$\begin{equation}
\begin{pmatrix}
     (255,255,0)&(255,255,0)&(255,255,0)&(255,255,0)&(255,255,0)&(255,255,0)\\
     (255,255,0)&(0,0,255)&(255,255,0)&(255,255,0)&(0,0,255)&(255,255,0)\\
     (255,255,0)&(0,0,255)&(255,255,0)&(255,255,0)&(0,0,255)&(255,255,0)\\
     (255,255,0)&(255,255,0)&(255,255,0)&(255,255,0)&(255,255,0)&(255,255,0)\\
     (255,255,0)&(255,0,0)&(255,0,0)&(255,0,0)&(255,0,0)&(255,255,0)\\
     (255,255,0)&(255,255,0)&(255,255,0)&(255,255,0)&(255,255,0)&(255,255,0)
\end{pmatrix}\\
\text{Représentation matricielle de notre smiley en couleurs}
\end{equation}$  

![Smiley_2](../img/smiley_color.png "Smiley en couleur")

Contrairement à la matrice utilisée pour l'image en noir et blanc, celle-ci contient des tuples de 3 valeurs au lieu de valeurs unique par pixel.  
Le code Python permettant de générer cette image est le suivant :  

```python
import numpy as np

smiley_color = np.array([[(255,255,0),(255,255,0),(255,255,0),
                          (255,255,0),(255,255,0),(255,255,0)],
                         [(255,255,0),(0,0,255),(255,255,0),
                          (255,255,0),(0,0,255),(255,255,0)],
                         [(255,255,0),(0,0,255),(255,255,0),
                          (255,255,0),(0,0,255),(255,255,0)],
                         [(255,255,0),(255,255,0),(255,255,0),
                          (255,255,0),(255,255,0),(255,255,0)],
                         [(255,255,0),(255,0,0),(255,0,0),
                          (255,0,0),(255,0,0),(255,255,0)],
                         [(255,255,0),(255,255,0),(255,255,0),
                          (255,255,0),(255,255,0),(255,255,0)]])
plt.imshow(smiley_color)
plt.axis("off")
plt.savefig("smiley_color.png");
```  

## Quelles différences y a-t-il entre une image au format .tif et des formats d’images plus communs comme le .jpeg ? Quel intérêt d’utiliser le premier format en microscopie ?

Dans le cadre de notre analyse d’image nous disposons d'une image de microscopie à fluorescence au format .tif.  

Une image peut être stockée sous différents formats comme par exemple les formats : .jpeg, .png, .gif et .tif, etc. La principale différence entre le format .tif et les autres formats qui sont des formats plus communément utilisés est la suivante.  
Le format TIFF ne présente pas de compression, dans le cas ou il y aurait une compression cela n’impacte pas la qualité de l’image observée. Ce manque de compression implique que la taille du fichier sera élevée. Le format .tif est donc utilisé pour avoir des images de haute résolution/qualité qui permet d’avoir des images avec des détails important ce qui est idéal pour observer des images de microscopie.  
A contrario, le format .jpeg entraine une compression, tandis que l'algorithme de compression utilisé par le format .jpegs entraine une perte de données irréversible. Le format .png ressemble au format .jpeg sans perte de données lors de la compression, il a été initialement inventé pour le transfert d'image sur internet. 




## Comment charger une image à l'aide de scikit-image ?  

Le module *scikit-image* propose des fonctions pour la lecture et l'écriture d'images. Ces fonctions sont regroupées dans le sous-module `io`. En particulier, la fonction `imread()` permet de charger une image à partir d'un fichier.

Ici, nous chargeons le logo de *scikit-image* (stocké dans `logo`).

```python
import skimage.io

url_logo = "https://upload.wikimedia.org/wikipedia/commons/3/38/Scikit-image_logo.png"
logo = skimage.io.imread(url_logo)
```

## Comment est représenté une image chargée avec la fonction `imread()` ?

```python
print(type(logo))
```
```
>>> <class 'numpy.ndarray'>
```

L'image importée est stockée dans un tableau `numpy`. Il faudra donc importer le module `numpy` pour la manipuler.

```python
print(logo.shape)
logo[0]
```
```
>>> (500, 500, 4)
>>> array([[255, 255, 255, 255],
       [255, 255, 255, 255],
       [255, 255, 255, 255],
       ...,
       [255, 255, 255, 255],
       [255, 255, 255, 255],
       [255, 255, 255, 255]], dtype=uint8)
```

Le tableau `numpy` de `logo` est tridimensionnel : les deux premières dimensions correspondent aux coordonées (ligne, colonne), et la troisième correspond aux différents canaux. Chaque pixel est donc représenté *via* 4 valeurs : sa valeur dans les canaux rouge, vert, bleu, et alpha (pour l'opacité).

## Comment afficher une image dans Python ?

L'image peut être affichée avec n'importe quelle fonction qui sait lire un tableau `numpy` pour afficher une image. Ici, nous utilisons la fonction `imshow()` du module `pyplot` (bibliothèque matplotlib).  

```python
import matplotlib.pyplot as plt

plt.imshow(logo)
```  

![logo](https://upload.wikimedia.org/wikipedia/commons/3/38/Scikit-image_logo.png "logo scikit-image")

## Comment choisir les canaux ?

`logo` étant un tableau `numpy`, la sélection des canaux se fait plus simplement grâce à l'indiçage de la troisième dimension : `0` pour récupérer le canal rouge, `1` pour récupérer le vert, `2` pour récupérer le bleu, et `3` pour récupérer l'alpha (opacité).

```python
canal_rouge = logo[:, :, 0]
canal_vert = logo[:, :, 1]
canal_bleu = logo[:, :, 2]
canal_opacite = logo[:, :, 3]
```  

Afin d'afficher les différents canaux sélectionnés, on utilise des fonctions du module ```matplotlib.pyplot```.

```python
fig, subplots = plt.subplots(nrows=1, ncols=4, figsize=(20, 4))

(subp_rgb, subp_r, subp_g, subp_b) = subplots

subp_rgb.imshow(logo)
subp_rgb.set_title("all channels")
subp_rgb.axis("off")

subp_r.imshow(canal_rouge, cmap="gray")
subp_r.set_title("red channel")
subp_r.axis("off")

subp_g.imshow(canal_vert, cmap="gray")
subp_g.set_title("green channel")
subp_g.axis("off")

subp_b.imshow(canal_bleu, cmap="gray")
subp_b.set_title("blue channel")
subp_b.axis("off")

plt.savefig("logo_channels.png")
```  
  
  
![canaux_logo](../img/logo_channels.png "Visualisatiion des cannaux")

La sélection d'un canal réduit donc le tableau d'une dimension. On obtient alors un tableau `numpy` en bidimensionnel. Il est donc nécessaire d'indiquer à `imshow()` que nous sommes en niveaux de gris et non plus en RGBA. Ainsi, plus la couleur du pixel se rapproche du blanc et plus la couleur associée au canal est d'intensité élevée, comme on le constate sur cet exemple.

## Autres fonctions utiles pour la manipulation d'image 

- `exposure_histogram()` :  

Fonction renvoyant deux tableaux numpy. Un des tableaux numpy contenant les valeurs des pixels et le second contenant le comptage associé. Ce sont grâce à ces deux tableaux que nous pourrons construire l'histogramme des valeurs des pixels de l'image, nous permettant ensuite de défnir un seuil de sélection pour la segmentation.  

- `filter_gaussian()` :   

Matrice permettant de "lisser" l'image en recalculant les valeurs des pixels selon leurs voisins. En effet si les voisins d'un pixel ont des valeurs plus élevées que celui-ci alors sa valeur sera réévaluée à la hausse. Réciproquement, si les voisins d'un pixel ont des valeurs mois élevées que celui-ci alors sa valeur sera réévaluée à la baisse.  

- `mesure_labels()` :  

Fonction renvoyant le nombre d'objets segmentés ainsi que la matrice labelisée associée à la segmentation. Lorsque nous parlons de labeliser chaque objets issus de la segmentation reçoit un label et chaque pixel est associé à un label. Donc chaque pixel appartient à un objet ou n'appartient à aucun objet (avec la valeur 0).  

- `mesure_regions_props()` :   

Fonction permettant de réaliser diverses mesures sur les objets segmentés auparavant, tel que l'aire, le périmètre, l'intensité moyenne, ect.

- `measure.find_contours()` :  

Fonction renvoyant une matrice des coordonnées du contour des objets segmentés que l'on utilisera pour afficher la délimitation de l'objet sur l'image.  

### Fonctions utilisées pour créer une visualisation interactive de l'image

- `plotly.express.imshow()` :  

Fonction permettant d'afficher l'image à visualiser.  

- `.update_layout()` :  

Méthode pour redimensionner le graphique interactif.

- `.add_traces()` :  

Méthode permettant d'ajouter les objets segmentés sur le graphique interactif.

- `plotly.graph_objects.Scatter()` :  

Fonction permettant de créer l'aspect interactif du graphique, en créant une fenêtre indiquant les mesures réalisées sur les objets, ainsi en créant un contour autour des objets.

## Ressources

- Vidéo Scipy 16/06/2019 : [Image Analysis in Python with SciPy and Scikit Image](https://www.youtube.com/watch?v=d1CIV9irQAY), consulté le 15/10/2021.  

Ressource contruite sous la forme d'un cours interactif. Cela permet de comprendre le module car on peut suivre et reproduire les démonstrations sur notre machine. Malheureusement, certaines fonctions et commmandes ne fonctionnent plus de la même manière aujourd'hui. En effet, ils utilisent la version 0.15, alors que la version la plus récente au moment de la création de notre environement de travail (09/10/2021) est la version 0.18.3.  

- TD d’analyse d’image de microscopie, 2017, Griffin Chure: [TD Caltech](https://bi1.caltech.edu/code/t04_quantitative_image_processing.html), consulté le 17/10/2021.  

Ressource trés pertinente, puisqu'il s'agit d'une analyse d'image en microscopie à fluorecsence d'_E.coli_ semblable à notre projet. Malheureusement, les données permettant de reproduire les résultats observés dans ce TD ne sont plus accessibles, nous empêchant de reproduire ce TD.  


- [GitHub de scikit-image](https://github.com/scikit-image), consulté le 16/10/2021.  

Dépot GitHub du module `scikit-image`. Il propose plusieurs tutoriels et démonstrations du module. Sont présent des explications portant sur : la segmentation, le filtrage d'une image par exemple.

- Site web de la bibliothèque [scikit-image](https://scikit-image.org), consulté le 10/10/2021.  

Site du module, incluant toute la documantation du module et de ses fonctions. Ainsi qu'un petit nombre de tutoriels qui pourraient nous être utile au cours de notre projet. Site qui nous a été notamment utile pour comprendre le fonctionnement du filtre gaussien. 

- Site web : (https://scipy-lectures.org/packages/scikit-image/index.html), consulté le 16/10/2021.  

Tutoriels courts, permettant la prise en main du module `scikit-image`. Ce qui nous a permis de comprendre comment est représenté une image en Python.  

- [Forum sur image.sc](https://forum.image.sc/tag/scikit-image), consulté le 17/10/2021.  

Forum destiné uniquement au module `scikit-image`, qui sera probablement utile lors d'erreurs d'utilisation des fonctions du module.  

- [Publication (van der Walt *et al*., 2014)](http://dx.doi.org/10.7717/peerj.453), consulté le 15/10/2021.  

Publication du module, avec présence d'exemples d'utilisation du module dans différents domaines d'applications. Ces différents domaines sont principalement : l'éducation, la recherche et l'industrialisation.  

- Documentation officielle de [Plotly](https://plotly.com/python/), consulté le 12/11/2021.  

Site de la documentation officielle de Plotly, utilisée pour l'implémentation de la visualisation interactive des mesures effectuées sur les noyaux de l'image de microscopie.